In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
weight = data_train.groupby('item_id')['sales_value'].sum().reset_index()

In [6]:
weights = (weight['sales_value'])/sum(weight['sales_value'])
# weights = np.log(weight['sales_value']))

In [7]:
# list(zip(items, weights))
weight.head(5)

,item_id,sales_value
0,25671,20.94
1,26081,0.99
2,26093,1.59
3,26190,1.54
4,26355,1.98


Проверим сумму весов - должно равняться == 1

In [32]:
round(weights.sum())

1

In [9]:
items = weight['item_id']
# items = data_train.item_id.unique()

In [10]:
tuples = list(zip(items, weights))
items_weights = pd.DataFrame(tuples,
                  columns = ['item_id', 'weight'])

In [11]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items=(items_weights['item_id'])
    weights=(items_weights['weight'])
    
    recs = np.random.choice(items ,p=weights, size=n,replace=False)
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [12]:
%%time



result['weighted_random_recommendation'] = result['user_id'].map(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

CPU times: user 2.53 s, sys: 0 ns, total: 2.53 s
Wall time: 2.53 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[962568, 1043730, 93700, 837270, 1008212]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12427353, 7409698, 971949, 10198378, 6904495]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5, Precision@3, Recall@5 для каждого алгоритма с помощью функций из вебинара 1. Какой алгоритм показывает лучшее качество?

In [26]:
result.head(2)

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[962568, 1043730, 93700, 837270, 1008212]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12427353, 7409698, 971949, 10198378, 6904495]"


In [16]:
# your_code
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [23]:
# your_code
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    recall_at_k = flags.sum() / len(bought_list)
    
    
    return recall_at_k

In [24]:
result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual'],k=5), axis=1).mean()

0.022918707149853133

In [22]:
result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual'],k=3), axis=1).mean()

0.024649036891936045

In [25]:
result.apply(lambda row: recall_at_k(row['weighted_random_recommendation'], row['actual'],k=5), axis=1).mean()

0.0036964700686015926

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 (или другое количество) товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [15]:
# your_code